## Libraries

In [29]:
import sys
import os
from requests.auth import HTTPProxyAuth

# Obtener la ruta del directorio actual del notebook
notebook_dir = os.getcwd()

# Añadir el directorio principal 'project' al path
sys.path.append(os.path.abspath(os.path.join(notebook_dir, '..')))

import config
from config import PROXI_USER, PROXI_PWD, EMAIL, SUBJECT, PROXIES, API_TOKEN
import pandas as pd

import requests
from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json

from bs4 import BeautifulSoup
from tqdm import tqdm

from requests_ntlm import HttpNtlmAuth


## Data Collection

### NO USAR

In [2]:
#si se incluye mas de una fecha o serie, incluir /
fecha = '2024-10-04'
serie = 'SF43718'

#url = "https://www.banxico.org.mx/SieAPIRest/service/v1/series/"+serie+"/datos/"+fecha+"/"+fecha+""

headers = {
    'Bmx-Token': '80f8b40305f8c5067445bd8868c2be2fd29f526c597fbadffa7cca63509a25a7',  # Cambia esto por el header que necesites
    'Content-Type': 'application/json',
}

proxies = PROXIES
auth = HTTPProxyAuth(PROXI_USER, PROXI_PWD)


### Precio Dollar

In [3]:
#API_TOKEN = '80f8b40305f8c5067445bd8868c2be2fd29f526c597fbadffa7cca63509a25a7'
url_banxico = 'https://www.banxico.org.mx/SieAPIRest/service/v1/series/SF43787,SF43784/datos/oportuno?token='+API_TOKEN

In [4]:
try:
    #response = requests.get(url,  proxies=proxies, auth=auth, headers=headers)
    response = requests.get(url_banxico)
    
    if response.status_code == 200:
        dataDolar = response.json()
    else:
        print(f"Error en Peticio: {response.status_code}")
        print(f"Detalles del error: {response.text}")
except requests.exceptions.RequestException as e:
    print(f"Request failed: {e}")
    #print(f"Error: {response.status_code}, {response.text}")


### Precio Crypto-Monedas

In [5]:
urlCriptos = 'https://api.coingecko.com/api/v3/coins/markets?vs_currency=usd&order=market_cap_desc&per_page=100&page=1&sparkline=false'


In [6]:
try:
    #response = requests.get(url,  proxies=proxies, auth=auth, headers=headers)
    response = requests.get(urlCriptos)
    
    if response.status_code == 200:
        dataCryptos = response.json()
    else:
        print(f"Error en Peticio: {response.status_code}")
        print(f"Detalles del error: {response.text}")
except requests.exceptions.RequestException as e:
    print(f"Request failed: {e}")
    #print(f"Error: {response.status_code}, {response.text}")

## Data Understanding

### Dolar

In [7]:
dataDolar.keys()

dict_keys(['bmx'])

### Previo Venta

In [8]:
#Venta
fechaVenta = dataDolar['bmx']['series'][0]['datos'][0]['fecha']
precioVenta = dataDolar['bmx']['series'][0]['datos'][0]['dato']

### Precio Compra

In [9]:
#Compra
#response['bmx']['series'][1]
fechaCompra = dataDolar['bmx']['series'][1]['datos'][0]['fecha']
precioCompra = dataDolar['bmx']['series'][1]['datos'][0]['dato']

### Crypto Monedas

In [10]:
dataCryptos[0]

{'id': 'bitcoin',
 'symbol': 'btc',
 'name': 'Bitcoin',
 'image': 'https://coin-images.coingecko.com/coins/images/1/large/bitcoin.png?1696501400',
 'current_price': 66228,
 'market_cap': 1308075121808,
 'market_cap_rank': 1,
 'fully_diluted_valuation': 1389620220006,
 'total_volume': 46079559044,
 'high_24h': 66173,
 'low_24h': 62465,
 'price_change_24h': 3445.64,
 'price_change_percentage_24h': 5.48819,
 'market_cap_change_24h': 65621740068,
 'market_cap_change_percentage_24h': 5.28163,
 'circulating_supply': 19767687.0,
 'total_supply': 21000000.0,
 'max_supply': 21000000.0,
 'ath': 73738,
 'ath_change_percentage': -10.43974,
 'ath_date': '2024-03-14T07:10:36.635Z',
 'atl': 67.81,
 'atl_change_percentage': 97291.03485,
 'atl_date': '2013-07-06T00:00:00.000Z',
 'roi': None,
 'last_updated': '2024-10-14T22:45:47.867Z'}

### Codigo

In [13]:
dataCryptos[0]['id']

'bitcoin'

### Nombre

In [15]:
dataCryptos[0]['name']

'Bitcoin'

### Precio


In [16]:
dataCryptos[0]['current_price']

66228

### Last Updated

In [26]:
dataCryptos[0]['last_updated'].split('T')[0]

'2024-10-14'

## Data Preparation

### Precio Dolar - Armado de Mensaje

In [26]:
mensaje = "\nHola, Informacion sobre tipo de Cambio:\nVenta:\n Fecha Venta: {}\n Valor: ${}\nCompra\n Fecha Compra: {}\n Valor: ${}".format(fechaVenta, precioVenta, fechaCompra, precioCompra)
print(mensaje)


Hola, Informacion sobre tipo de Cambio:
Venta:
 Fecha Venta: 14/10/2024
 Valor: $19.3580
Compra
 Fecha Compra: 14/10/2024
 Valor: $19.3490


### Precio Crypto Monedas

In [31]:
def get_crypto(data, i):
    id = dataCryptos[i]['id']
    name = dataCryptos[i]['name']
    price = dataCryptos[i]['current_price']
    dateUpdate = dataCryptos[i]['last_updated'].split('T')[0]
    timeUpdate = dataCryptos[i]['last_updated'].split('T')[1]
    

    return id, name, price, dateUpdate, timeUpdate
    

In [34]:
data = []

for i in tqdm(range(0, 10), colour= 'blue'):
    data.append(get_crypto(dataCryptos, i))

100%|██████████| 10/10 [00:00<00:00, 14315.03it/s]


In [33]:
col = ['Id', 'Name', 'Current_Price', 'Date_Updated', 'Time_Updated']
df_cryptos = pd.DataFrame(data, columns=col)

df_cryptos

,Id,Name,Current_Price,Date_Updated,Time_Updated
0,bitcoin,Bitcoin,66228.000000,2024-10-14,22:45:47.867Z
1,ethereum,Ethereum,2637.120000,2024-10-14,22:45:58.064Z
2,tether,Tether,1.001000,2024-10-14,22:45:52.128Z
3,binancecoin,BNB,589.880000,2024-10-14,22:45:46.026Z
4,solana,Solana,157.060000,2024-10-14,22:45:53.477Z
5,usd-coin,USDC,1.000000,2024-10-14,22:45:53.727Z
6,ripple,XRP,0.548861,2024-10-14,22:45:56.285Z
7,staked-ether,Lido Staked Ether,2635.850000,2024-10-14,22:45:50.618Z
8,dogecoin,Dogecoin,0.116993,2024-10-14,22:45:47.622Z
9,tron,TRON,0.160960,2024-10-14,22:45:52.521Z
